<a href="https://colab.research.google.com/github/Interactions-SpoofProof/interactions-ai-studio-project/blob/main/ver2_neural_network_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Steps for preparing the feature vectors for the neural network:
1. Make all vectors the same size by truncating/padding with 0's so that dimensions are (1, 300, 768)
2. Use normalization/standardization (I used standardization) (this is where preprocessed_feature_vectors.pkl ended)
3. Reduce the embedding dimensions (768 -> 256) using PCA (this is where reduced_feature_vecs.pkl.gz ended)
4. Use squeeze function to remove the 1 from the dimensions
5. Flatten the feature vectors into 1D arrays since neural networks only accept 2D arrays in the form (num_samples, num_features)

Note: I used GPU to train model so only took a few mins

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gzip
import pickle

file_path = '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/test_dataset_compressed.pkl.gz'

with gzip.open(file_path, 'rb') as f:
    test_feature_vecs = pickle.load(f)

In [ ]:
test_feature_vecs[0].shape

(1, 177, 768)

In [ ]:
import gzip
import pickle
file_path = '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/test_dataset_compressed.pkl.gz'

with gzip.open(file_path, 'rb') as f:
    test_dataset = pickle.load(f) # contains 4 cols: file, audio, label, number_array

In [ ]:
# step 1: padding with 0's (10 secs)
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Pad all sequences to 300
padded_vecs = []
for i in range(len(test_feature_vecs)):
  padded_feature_vecs = pad_sequences(test_feature_vecs[i], maxlen=300, dtype='float32', padding='post', value=0.0)

  #print("Padded feature vectors shape:", padded_feature_vecs.shape)
  padded_vecs.append(padded_feature_vecs)

In [ ]:
padded_vecs[0].shape

(1, 300, 768)

In [ ]:
# step 2: standardization (10 secs)
from sklearn.preprocessing import StandardScaler

standardized_vecs = []

for i in range(len(padded_vecs)):

# Assuming padded_vecs[i] is a 3D numpy array: (num_samples, num_frames, embedding_dim)
  num_samples, num_frames, embedding_dim = padded_vecs[i].shape

# Reshape the features to 2D for standardization
  flattened_features = padded_vecs[i].reshape(num_samples * num_frames, embedding_dim)

# Apply standardization
  scaler = StandardScaler()
  standardized_features = scaler.fit_transform(flattened_features)

# Reshape back to 3D
  standardized_features = standardized_features.reshape(num_samples, num_frames, embedding_dim)
  standardized_vecs.append(standardized_features)


In [ ]:
standardized_vecs[0].shape

(1, 300, 768)

In [ ]:
# step 3: Reduce the embedding dimensions from (768 -> 256) using PCA (3 mins)
from sklearn.decomposition import PCA

reduced_feature_vecs = []

for i in range(len(standardized_vecs)):
  num_samples, num_frames, embedding_dim = standardized_vecs[i].shape

# Reshape to 2D (combine samples and frames into one dimension for PCA)
  flattened_features = standardized_vecs[i].reshape(num_samples * num_frames, embedding_dim)

# Perform PCA to reduce to 256 dimensions instead of 768
  pca = PCA(n_components=256)
  reduced_features = pca.fit_transform(flattened_features)

# Reshape back to 3D: (num_samples, num_frames, 256)
  reduced_feature_vec = reduced_features.reshape(num_samples, num_frames, 256)
  reduced_feature_vecs.append(reduced_feature_vec)

In [ ]:
reduced_feature_vecs[1].shape

(1, 300, 256)

In [ ]:
# step 4: Use squeeze function to remove the extra 1 from the dimensions (1 sec)
import numpy as np

feature_vecs = np.array(reduced_feature_vecs)
feature_vecs_squeezed = feature_vecs.squeeze(axis=1)

In [ ]:
feature_vecs_squeezed[0].shape

(300, 256)

In [ ]:
# don't use this code for flattening vectors
#flattened_feature_vecs = []

#for i in range(len(feature_vecs_squeezed)):
  #reduced_feature_vecs[i] = np.array(feature_vecs_squeezed[i])
  #flattened_feature_vec = feature_vecs_squeezed.reshape(feature_vecs_squeezed[i].shape[0], -1)
  #flattened_feature_vecs.append(flattened_feature_vec)

In [ ]:
# step 5: flatten feature vectors to 1D arrays instead of 2D
#flattened_feature_vecs = feature_vecs_squeezed.reshape(feature_vecs.shape[0], -1)
flattened_feature_vecs = feature_vecs_squeezed.reshape(feature_vecs_squeezed.shape[0], -1)
flattened_feature_vecs[0].shape

(76800,)

In [ ]:
import pandas as pd

test_df = pd.DataFrame(flattened_feature_vecs)
test_df['label'] = labels

In [ ]:
with gzip.open('/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/test_final_preprocessed_vectors.pkl.gz', 'wb') as f:
    pickle.dump(test_df, f)

In [ ]:
import pandas as pd

#dataframe with the flattened feature vectors
test_df = pd.DataFrame(flattened_feature_vecs)

# label added as a new column in test_df
labels = test_dataset['label'].to_list()
test_df['label'] = labels

In [ ]:
label_col = test_df.pop('label')
test_df.insert(0, 'label', label_col)  # setting label as leftmost column

In [ ]:
X = test_df.drop('label', axis=1)
y = test_df['label']

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
import tensorflow as tf

# Check if GPU is available
print("GPU Available: ", tf.test.is_gpu_available())


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU Available:  True


In [ ]:
from keras.layers import Dense, Input
# Simplified model with smaller hidden layers
model = Sequential()

# Use Input layer for the first layer to specify input shape
model.add(Input(shape=(X.shape[1],)))  # Specify the input shape here

model.add(Dense(32, activation='relu'))  # Reduce number of neurons
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))  # Binary classification output

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# fit model to data (40 sec)
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2, shuffle=True)  # Adjust epochs and batch_size as needed


Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 114ms/step - accuracy: 0.3717 - loss: 3.0335 - val_accuracy: 0.8325 - val_loss: 0.3986
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.6029 - loss: 2.1979 - val_accuracy: 0.9000 - val_loss: 0.2170
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.6759 - loss: 1.9279 - val_accuracy: 0.9300 - val_loss: 0.1643
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.7171 - loss: 1.6667 - val_accuracy: 0.9375 - val_loss: 0.1482
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.7548 - loss: 1.0674 - val_accuracy: 0.9150 - val_loss: 0.1639
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.8049 - loss: 0.9876 - val_accuracy: 0.9300 - val_loss: 0.1857
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.7931 - loss: 0.9287 - val_accuracy: 0.9425 - val_loss: 0.1722
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - accuracy: 0.7953 - loss: 0.7579 - val_accuracy: 0.9125 - 

In [ ]:
# evaluating model
score = model.evaluate(X, y, verbose=0)
print(f'Test Loss: {score[0]}, Test Accuracy: {score[1]}')


Test Loss: 0.09159397333860397, Test Accuracy: 0.9635000228881836
